Belki ileri de daha `genel` bir dökümantasyon hazırlanması gerekirse, tüm `global` değişkenleri de `fonksiyon argümanı olarak atayabiliriz.

In [1]:
using QuantumOptics

# https://juliapackages.com/p/einsum
using Einsum

In [2]:
"""
Compute the single-particle operator from single-particle matrix.

#### Arguments
- `sp_basis::NLevelBasis`: Single-Particle basis.
- `sp_matrix::Matrix`: Any hopping phases matrix from any model.
"""
function get_sp_op(sp_basis, sp_matrix)
  
    H = SparseOperator(sp_basis)

    for m in 1:N
        for n in 1:N
            H += sp_matrix[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

get_sp_op

In [ ]:
function get_sp_op_data(sp_basis, sp_matrix)
  
    H = SparseOperator(sp_basis)

    for m in 1:N
        for n in 1:N
            H += sp_matrix.data[m,n] * transition(sp_basis, m, n)
        end
    end
    
    return H
end

In [3]:
"""
Compute the first given number eigen-states.

#### Arguments
- `sp_op::Operator`: Created single-particle operator by using `get_sp_op`.
- `cut_off::Int`: Eigen-states order from `1` to `cut_off`.
"""
function get_sub_states(sp_op, cut_off)
    
    E0, states0 = eigenstates(dense(sp_op))
    states = states0[1:cut_off]
    
    return states
end

get_sub_states

In [4]:
"""
Compute Sub-Space Basis, Projection and comlex conjugate of Projection operator.

#### Arguments
- `states::eigenstates`: Eigen-states of any operator.
- `basis::NLevelBasis`: Single-Particle basis.
"""
function get_projector_op(states, basis)
    
    b_sub = SubspaceBasis(basis,states)
    P = projector(b_sub, basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_projector_op

In [5]:
"""
Compute the Sub-Space Operator.

#### Arguments
- `sp_op::Operator`: Single-particle operator from single-particle matrix.
- `P::Operator`: Projection operator.
- `Pt::Operator`: Complex conjugate of Projection operator.
"""
function get_subspace_op(sp_op, P, Pt)
    return P*sp_op*Pt
end

get_subspace_op

In [6]:
"""
Compute the Single-Particle Number Operator for each lattice sites.

#### Arguments
- `sp_basis::NLevelBasis`: Single-Particle basis.
- `P::Operator`: Projection operator.
- `Pt::Operator`: Complex conjugate of Projection operator.
"""
function get_num_sub_list(sp_basis, P, Pt)
    num_sub_list = []
    for m in 1:N
        NM = transition(sp_basis, m, m)
        NMP = get_subspace_op(NM, P, Pt)
        push!(num_sub_list, NMP)
    end
    return num_sub_list
end

get_num_sub_list

In [7]:
"""
Compute the Many-Body Operator from Single-Particle Operator.

#### Arguments
- `basis_mb::ManyBodyBasis`: Created Many-Body Basis by using `bosonstates` or `fermionstates`.
- `basis::SubSpaceBasis`: Created Sub-Space Basis by using get_projector_op.
- `sp_op::Operator`: Single-particle operator from single-particle matrix.
"""
function get_mb_op(basis_mb, basis, sp_op)
    
    Op_MB = SparseOperator(basis_mb)
    
    for i in 1:length(basis)
        for j in 1:length(basis)
            Op_MB += sp_op.data[i,j] * transition(basis_mb, i, j)
        end
    end
    
    return Op_MB
end

get_mb_op

In [8]:
"""
Compute the Many-Body Number Operator for each lattice sites.

#### Arguments
- `basis_mb::ManyBodyBasis`: Created Many-Body Basis by using `bosonstates` or `fermionstates`.
- `basis::SubSpaceBasis`: Created Sub-Space Basis by using get_projector_op.
- `num_sub_list::ElementVector`: Created Single-Particle Number Number Operator for each lattice sites by using get_num_sub_list.
"""
function get_num_mb_list(basis_mb, basis, num_sub_list)
    
    num_mb_list = []
    
    for m in 1:N
        NMP = get_mb_op(basis_mb, basis, num_sub_list[m])
        push!(num_mb_list, NMP)
    end
    
    return num_mb_list
end

get_num_mb_list

In [9]:
function get_hubbard_int(num_op_list, basis_mb, U)

    IT = SparseOperator(basis_mb)
    
    for m in 1:N
        IT += U/2 * ( num_op_list[m] * num_op_list[m] - num_op_list[m] )
    end
    
    return IT
end

get_hubbard_int (generic function with 1 method)

In [10]:
function get_hubbard_int2(basis, P, Pt)

    basis2 = basis ⊗ basis

    # interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i

    Vint = SparseOperator(basis2)

    for n in 1:N
        Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
    end

    Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

    Vint_mb = manybodyoperator(basis_mb, Vint_sub)
    
    return Vint_mb
end

get_hubbard_int2 (generic function with 1 method)

In [11]:
function get_hubbard_int3(P, Pt, b_sub, cut_off)
   
    P1 = P.data
    P1t = Pt.data

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    b2_sub = b_sub ⊗ b_sub

    P4re = reshape(P4, cut_off^2, cut_off^2)

    Vint_bsub2 = SparseOperator(b2_sub,U/2*P4re)
    
    Vint_bsub2_mb = manybodyoperator(basis_mb, Vint_bsub2)
    
    return Vint_bsub2_mb
end

get_hubbard_int3 (generic function with 1 method)

In [12]:
function get_hubbard_int4(P, Pt, b_sub, cut_off)
    
    bcut_mb, bcut = get_Bosonic_MB_Basis(cut_off, PN)

    P1 = P.data
    P1t = Pt.data;

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    b2cut = bcut ⊗ bcut

    P4re = reshape(P4, cut_off^2, cut_off^2)

    Vint_bsub2 = SparseOperator(b2cut, U/2*P4re)
        
    Vint_mb_cut = manybodyoperator(bcut_mb, Vint_bsub2)
    
    return Vint_mb_cut
end

get_hubbard_int4 (generic function with 1 method)

In [13]:
"""
Compute the on-site Bose-Hubbard Interaction term.

#### Arguments:
- `P::Operator`: Projection operator.
- `Pt::Operator`: Complex conjugate of Projection operator.
- `b_sub::SubSpaceBasis`: Created Sub-Space Basis by using get_projector_op.
- `cut-off::Integer`: A Integer variable.
"""
function Hubbard_Interaction(P, Pt, b_sub, cut_off)
    
    bcut_mb, bcut = get_Bosonic_MB_Basis(cut_off, PN)
    
    P1 = P.data
    P1t = Pt.data;

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    Vint_mb_cut = SparseOperator(bcut_mb)
        
    for k in 1:cut_off
        for l in 1:cut_off
            for m in 1:cut_off
                for n in 1:cut_off
                    a1t = create(bcut_mb, k)
                    a2t = create(bcut_mb, l)
                    a2  = destroy(bcut_mb, m)      
                    a1  = destroy(bcut_mb, n)      
                    Vint_mb_cut += U/2*P4[k,l,m,n]*a1t*a2t*a2*a1
                end
            end
        end
    end
    
    return Vint_mb_cut
end

Hubbard_Interaction